In [51]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
root_dir='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/scripts_hongli/'
os.chdir(root_dir)
output_folder='step5_water_balance'
text_outputfile = 'annual_water_balance.txt'
plot_outputfile = 'annual_water_balance.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

sim_file='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/output/gmet_3hr/output_output_day.nc'
var_names = ['pptrate_mean', 'airtemp_mean', 'scalarCanopyWat', 'scalarSWE', 'scalarAquiferStorage', 'scalarTotalSoilWat',
            'scalarSurfaceRunoff_mean', 'scalarSoilBaseflow_mean', 'scalarAquiferBaseflow_mean', 
             'scalarTotalET_mean', 'scalarTotalRunoff_mean']
var_units = ['kg m-2 s-1', 'K', 'kg m-2', 'kg m-2','m', 'kg m-2', 'm s-1', 'm s-1', 'm s-1', 'kg m-2 s-1', 'm s-1']

# surf_runoff_vname = 'scalarSurfaceRunoff_mean' #'m s-1'
# base_flow_vname = 'scalarSoilBaseflow_mean' #'m s-1'

p_vname = 'pptrate_mean' #'kg m-2 s-1'
et_vname = 'scalarTotalET_mean' #'kg m-2 s-1' 
runoff_vname = 'scalarTotalRunoff_mean' #'m s-1'
s1_vname = 'scalarCanopyWat' #'kg m-2'
s2_vname = 'scalarSWE'
s3_vname = 'scalarTotalSoilWat' 

t_vname = 'time'
time_format='%Y-%m-%d'

dpi_value = 150

# read data
print('read')
f = xr.open_dataset(sim_file)
var_list = f.variables
sim_time_str = f[t_vname].dt.strftime(time_format).values[:]
sim_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in sim_time_str])
wateryear_list = np.unique([t.year for t in sim_time])[1:]

# surf_runoff_value = f[surf_runoff_vname].values[:]*1000*86400 # m/s to mm/day
# base_flow_value = f[base_flow_vname].values[:]*1000*86400 # m/s to mm/day

p_value = f[p_vname].values[:]*86400 # kg/m^2/s to mm/day
et_value = f[et_vname].values[:]*86400 # kg/m^2/s to mm/day 
runoff_value = f[runoff_vname].values[:]*1000*86400 # m/s to mm/day
s1_value = f[s1_vname].values[:] #kg/m-2 = mm
s2_value = f[s2_vname].values[:]
s3_value = f[s3_vname].values[:]

f.close()

# calculate
print('calculate')
hru_num = np.shape(p_value)[1]
for hru in range(hru_num):
    details = []
    for wateryear in wateryear_list:
        
        # identify wateryear corresponding index
        wateryear_ini = datetime.datetime.strptime(str(wateryear-1)+'-10-01', time_format)
        wateryear_end = datetime.datetime.strptime(str(wateryear)+'-09-30', time_format)        
        wateryear_index=[loc for loc in range(len(sim_time)) if (sim_time[loc]>= wateryear_ini and sim_time[loc]<=wateryear_end)]

        # calcualte annual cumulative flux
        p = np.sum(p_value[wateryear_index,hru])
        et = np.sum(et_value[wateryear_index,hru])
        runoff = np.sum(runoff_value[wateryear_index,hru])
        calclulate_delta_s = p+et-runoff #Note summa outputs ET with negative sign! 

        # calculate water storage
        s_ini = s1_value[wateryear_index[0],hru]+s2_value[wateryear_index[0],hru]+s3_value[wateryear_index[0],hru]
        s_end = s1_value[wateryear_index[-1],hru]+s2_value[wateryear_index[-1],hru]+s3_value[wateryear_index[-1],hru]
        actual_delta_s = s_end-s_ini

        # save annual data
        hru_year_data = [wateryear, p, et, runoff, s_ini, s_end, calclulate_delta_s, actual_delta_s]
        details.append(hru_year_data)

    # output
    file_name = text_outputfile.split('.')[0]+'_hru'+str(hru+1)+'.'+text_outputfile.split('.')[1]
    header_txt='Water year, P, ET, Runoff, Storage_initial, Storage_end, Qin - Qout, End - Initial storage' 
    var_num = len(hru_year_data)
    np.savetxt(os.path.join(root_dir, output_folder, file_name), details, header=header_txt, fmt='%d, ' + '%.4f, '*(var_num-1))

# plot
print('plot')
col_num=3 
row_num=3             
fig, ax = plt.subplots(row_num,col_num)
fig.set_figwidth(6.5*0.6*row_num) #190mm
fig.set_figheight(6.5*0.6*0.65*col_num) #heigh/width=3/4

for k in range(row_num): # hru
    
    file_name = text_outputfile.split('.')[0]+'_hru'+str(k+1)+'.'+text_outputfile.split('.')[1]
    data = np.loadtxt(os.path.join(root_dir, output_folder, file_name), delimiter=',', skiprows=1, usecols=range(1,var_num))
    
    for j in range(col_num): 
        
        if j == 0: #Qin, Qout
            ax[k,j].plot(wateryear_list, data[:,0], 'b^-', linewidth=1.0, markersize=1.0, alpha=0.8, label='Precipitation')
            ax[k,j].plot(wateryear_list, data[:,1]*(-1), 'rs--', linewidth=1.0, markersize=1.0, alpha=0.8, label='Evapotranspiration')
            ax[k,j].plot(wateryear_list, data[:,2], 'ko-.', linewidth=1.0, markersize=1.0, alpha=0.8, label='Runoff')
        elif j == 1: #Storage initial, Storage end
            ax[k,j].plot(wateryear_list, data[:,3], 'b^-', linewidth=1.0, markersize=1.0, alpha=0.8, label='Initial S')
            ax[k,j].plot(wateryear_list, data[:,4], 'rs--', linewidth=1.0, markersize=1.0, alpha=0.8, label='End S')
        elif j == 2: #Calculated vs. actual storages
            ax[k,j].plot(wateryear_list, data[:,5], 'b^-', linewidth=1.0, markersize=1.0, alpha=0.8, label='Qin - Qout')
            ax[k,j].plot(wateryear_list, data[:,6], 'rs--', linewidth=1.0, markersize=1.0, alpha=0.8, label='End S - Initial S')
        
        plot_alpha = chr(ord('a')+k*col_num+j)
        if j == 0:
            title = '(' + plot_alpha + ') HRU'+ str(k+1) + ' Flux'
            ylabel = 'Q (mm)'
        elif j == 1:
            title = '(' + plot_alpha + ') HRU'+ str(k+1) + ' Storage'
            ylabel = 'S (mm)'
        elif j == 2:
            title = '(' + plot_alpha + ') HRU'+ str(k+1) + ' '+r'$\Delta$'+'Storage'
            ylabel = r'$\Delta$'+'S (mm)'
        ax[k,j].set_title(title, weight='semibold', fontsize='small')
        ax[k,j].set_ylabel(ylabel, fontsize='small')
        ax[k,j].yaxis.set_tick_params(labelsize='small')

        if k == row_num-1:
            ax[k,j].set_xlabel('Water year', fontsize='small')
            ax[k,j].xaxis.set_tick_params(labelsize='small')        
        ax[k,j].xaxis.set_tick_params(labelsize='small')        
        ax[k,j].legend(loc='upper right', fontsize='x-small', framealpha=0.5)

fig.tight_layout()
fig.savefig(os.path.join(root_dir, output_folder, plot_outputfile), dpi=dpi_value)
plt.close(fig)

read
calculate
plot
